In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
%reload_ext autoreload

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
from optimizers_niapy import *
from other_optimizers import *

# GCN results

In [6]:
model = GCNLinkPredictor(in_channels=5, hidden_channels=256)
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\n=== Before training ===")
test_probs = GCNtest(model, test_data)
print(f"Sample test predictions (first 5):\n{test_probs[:5]}")

print("\n=== After training ===")
for epoch in range(1, 10):
  loss = GCNtrain(model, optimizer, train_data)
  test_probs = GCNtest(model, test_data)
  auc_gcn, f1_gcn, threshold = evaluate_model(test_probs, test_data.edge_label)
  print(f"Epoch {epoch}, Loss: {loss:.4f}, AUC score: {auc_gcn:.4f}, F1 score: {f1_gcn:.4f}")
  print(f"Sample predictions: {test_probs[:5].tolist()}...")



=== Before training ===
Sample test predictions (first 5):
[0.530487   0.5121068  0.5320257  0.51800746 0.51751906]

=== After training ===
Epoch 1, Loss: 1.3755, AUC score: 0.7529, F1 score: 0.7278
Sample predictions: [0.756817102432251, 0.6171451210975647, 0.8041971921920776, 0.6594882011413574, 0.6854455471038818]...
Epoch 2, Loss: 1.4892, AUC score: 0.7374, F1 score: 0.7105
Sample predictions: [0.5396678447723389, 0.5136833786964417, 0.5454912185668945, 0.5203129053115845, 0.5249595046043396]...
Epoch 3, Loss: 1.3677, AUC score: 0.7192, F1 score: 0.7125
Sample predictions: [0.6969800591468811, 0.5894076824188232, 0.7471961975097656, 0.6259374618530273, 0.6321976780891418]...
Epoch 4, Loss: 1.4305, AUC score: 0.7214, F1 score: 0.7140
Sample predictions: [0.6559635996818542, 0.5706409811973572, 0.7004437446594238, 0.6001905798912048, 0.6043412089347839]...
Epoch 5, Loss: 1.3966, AUC score: 0.7446, F1 score: 0.7234
Sample predictions: [0.6085211038589478, 0.5484013557434082, 0.639589

# GAN results

In [8]:
generator = Generator(in_channels=5, hidden_channels=256).to(device)
discriminator = Discriminator(hidden_channels=256).to(device)

generator.apply(init_weights)
discriminator.apply(init_weights)

optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.999))

print("\n=== Before training ===")
test_probs = GANtest(generator, discriminator, test_data)
print(f"Sample test predictions (first 5):\n{test_probs[:5]}")

print("\n=== After training ===")
for epoch in range(1, 10):
    d_loss, g_loss = GANtrain(generator, discriminator, optimizer_G, optimizer_D, train_data)
    test_probs = GANtest(generator, discriminator, test_data)
    auc_gan, f1_gan, threshold = evaluate_model(test_probs, test_data.edge_label)
    print(f"Epoch {epoch}, D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}, AUC score: {auc_gan:.4f}, F1 score: {f1_gan:.4f}")
    print(f"Sample predictions: {test_probs[:5].tolist()}...")



=== Before training ===
Sample test predictions (first 5):
[[0.4967758 ]
 [0.51241285]
 [0.5010973 ]
 [0.49302885]
 [0.49735793]]

=== After training ===
Epoch 1, D Loss: 0.0077, G Loss: 0.0016, AUC score: 0.5059, F1 score: 0.6809
Sample predictions: [[0.49454465508461], [0.5098069310188293], [0.4993833899497986], [0.49015289545059204], [0.4931155741214752]]...
Epoch 2, D Loss: -0.0004, G Loss: 0.0192, AUC score: 0.5488, F1 score: 0.6845
Sample predictions: [[0.49249210953712463], [0.5075169801712036], [0.4976271390914917], [0.48761773109436035], [0.48919180035591125]]...
Epoch 3, D Loss: -0.0148, G Loss: 0.0415, AUC score: 0.5904, F1 score: 0.6886
Sample predictions: [[0.49038979411125183], [0.5052443742752075], [0.4957611560821533], [0.4850482642650604], [0.4851073920726776]]...
Epoch 4, D Loss: -0.0157, G Loss: 0.0549, AUC score: 0.6278, F1 score: 0.6972
Sample predictions: [[0.4884045422077179], [0.5030146241188049], [0.4941975474357605], [0.48277002573013306], [0.4815029799938202

# Results after optimization with Genetic Algorithm

## GCN

In [19]:
result = run_gcn_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 143
  Learning Rate   : 0.00448
  Num Layers      : 3
  Dropout         : 0.35

Performance Metrics:
  F1 Score        : 0.8479
  AUC             : 0.9154
  Average Loss    : 1.2360


## GAN

In [4]:
result = run_gcn_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 202
  Learning Rate   : 0.00629
  Dropout         : 0.06

Performance Metrics:
  F1 Score        : 0.8476
  AUC             : 0.5000
  Average Loss    : 31.2233


# Results after optimization with Particle Swarm Optimization

## GCN

In [5]:
result = run_gcn_pso()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 32
  Learning Rate   : 0.01579
  Num Layers      : 4
  Dropout         : 0.00

Performance Metrics:
  F1 Score        : 0.8498
  AUC             : 0.8287
  Average Loss    : 1.3822


## GAN

In [15]:
result = run_gan_pso()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 277
  Learning Rate   : 0.00308
  Dropout         : 0.45

Performance Metrics:
  F1 Score        : 0.7497
  AUC             : 0.5540
  Average Loss    : 0.0264


# Results after optimization with Simulated Annealing

## GCN

In [9]:
result = run_gcn_sa()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 112
  Learning Rate   : 0.03100
  Num Layers      : 3
  Dropout         : 0.55

Performance Metrics:
  F1 Score        : 0.8393
  AUC             : 0.8354
  Average Loss    : 1.3810


## GAN

In [16]:
result = run_gan_sa()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 242
  Learning Rate   : 0.00001
  Dropout         : 0.10

Performance Metrics:
  F1 Score        : 0.6804
  AUC             : 0.5000
  Average Loss    : -3.8527


# Results after optimization with Artificial Bee Colony

## GCN

In [12]:
result = run_gcn_abc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 32
  Learning Rate   : 0.02367
  Num Layers      : 3
  Dropout         : 0.48

Performance Metrics:
  F1 Score        : 0.8490
  AUC             : 0.5000
  Average Loss    : 31.2235


## GAN

In [17]:
result = run_gan_abc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 405
  Learning Rate   : 0.00276
  Dropout         : 0.47

Performance Metrics:
  F1 Score        : 0.7480
  AUC             : 0.5756
  Average Loss    : 0.0275


# Results after optimization with Ant Colony Optimization

## GCN

In [26]:
result = run_gcn_aco(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 64
  Learning Rate   : 0.01000
  Num Layers      : 4
  Dropout         : 0.00

Performance Metrics:
  F1 Score        : 0.8526
  AUC             : 0.9175
  Average Loss    : 1.2008


## GAN

In [29]:
result=run_gan_aco(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 512
  Learning Rate   : 0.00060
  Dropout         : 0.70

Performance Metrics:
  F1 Score        : 0.7490
  AUC             : 0.7728
  Average Loss    : 0.1738


# Results after optimization with Hill Climbing

## GCN

In [6]:
result = run_gcn_hc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 58
  Learning Rate   : 0.02508
  Num Layers      : 3
  Dropout         : 0.54

Performance Metrics:
  F1 Score        : 0.8491
  AUC             : 0.7861
  Average Loss    : 1.3760


## GAN

In [18]:
result = run_gan_hc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 398
  Learning Rate   : 0.00228
  Dropout         : 0.16

Performance Metrics:
  F1 Score        : 0.7494
  AUC             : 0.5000
  Average Loss    : -4.2903


# Results after optimization with Random Search

## GCN

In [7]:
result = run_gcn_ra()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 125
  Learning Rate   : 0.01720
  Num Layers      : 3
  Dropout         : 0.50

Performance Metrics:
  F1 Score        : 0.8411
  AUC             : 0.9035
  Average Loss    : 1.2390


## GAN

In [4]:
result = run_gan_ra()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 416
  Learning Rate   : 0.00258
  Dropout         : 0.32

Performance Metrics:
  F1 Score        : 0.7568
  AUC             : 0.7725
  Average Loss    : 0.0348


# Results after optimization with Bayesian Optimization

## GCN

In [20]:
result = run_gcn_bo(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 144
  Learning Rate   : 0.00264
  Num Layers      : 4
  Dropout         : 0.05

Performance Metrics:
  F1 Score        : 0.8524
  AUC             : 0.9179
  Average Loss    : 1.2434


## GAN

In [24]:
result = run_gan_bo(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 160
  Learning Rate   : 0.00062
  Dropout         : 0.35

Performance Metrics:
  F1 Score        : 0.7458
  AUC             : 0.7669
  Average Loss    : 0.1828


# Results after optimizing with Optuna

## GCN

In [16]:
result = run_gcn_optuna()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")

[I 2025-06-12 19:56:25,353] A new study created in memory with name: no-name-62abfe02-5699-4890-b902-1848d0bf8cec
[I 2025-06-12 19:56:54,306] Trial 0 finished with value: -0.7422966082908447 and parameters: {'hidden_channels': 228, 'lr': 0.00010650363018688238, 'num_layers': 4, 'dropout': 0.5561634941565968}. Best is trial 0 with value: -0.7422966082908447.
[I 2025-06-12 19:57:03,981] Trial 1 finished with value: -0.7941541520726247 and parameters: {'hidden_channels': 67, 'lr': 0.001985802358640859, 'num_layers': 3, 'dropout': 0.48122728482664434}. Best is trial 1 with value: -0.7941541520726247.
[I 2025-06-12 19:57:21,183] Trial 2 finished with value: -0.8475344185491959 and parameters: {'hidden_channels': 137, 'lr': 0.015525247915454551, 'num_layers': 4, 'dropout': 0.5794923157284809}. Best is trial 2 with value: -0.8475344185491959.
[I 2025-06-12 19:58:00,210] Trial 3 finished with value: -0.7809033708836948 and parameters: {'hidden_channels': 158, 'lr': 0.0001800540053010709, 'num_


Best Hyperparameters Found:
  Hidden Channels : 45
  Learning Rate   : 0.00698
  Num Layers      : 4
  Dropout         : 0.29

Performance Metrics:
  F1 Score        : 0.8519
  AUC             : 0.9184
  Average Loss    : 1.2382


## GAN

In [25]:
result = run_gan_optuna()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")

[I 2025-06-12 20:46:41,312] A new study created in memory with name: no-name-3e5781ab-4e10-4ab9-b7c8-f9434391b2cf
[I 2025-06-12 20:47:24,826] Trial 0 finished with value: -0.7402626523401754 and parameters: {'hidden_channels': 510, 'lr': 5.430084496031334e-05, 'dropout': 0.12647607665839158}. Best is trial 0 with value: -0.7402626523401754.
[I 2025-06-12 20:47:52,476] Trial 1 finished with value: -0.7410790092909298 and parameters: {'hidden_channels': 276, 'lr': 0.00021313926909560517, 'dropout': 0.5400292282797245}. Best is trial 1 with value: -0.7410790092909298.
[I 2025-06-12 20:48:06,673] Trial 2 finished with value: -0.6666666666666666 and parameters: {'hidden_channels': 169, 'lr': 0.015543512682181305, 'dropout': 0.0010090038802316292}. Best is trial 1 with value: -0.7410790092909298.
[I 2025-06-12 20:48:18,878] Trial 3 finished with value: -0.6994442335986525 and parameters: {'hidden_channels': 105, 'lr': 0.0024567710388621908, 'dropout': 0.168138006907391}. Best is trial 1 with


Best Hyperparameters Found:
  Hidden Channels : 373
  Learning Rate   : 0.00072
  Dropout         : 0.28

Performance Metrics:
  F1 Score        : 0.7499
  AUC             : 0.7727
  Average Loss    : 0.1771
